## Reverend Bayes' legendary coin of wisdom
### A parable

An eminent Professor of History at your university has recently discovered a relic that was once owned by the venerable Reverend Bayes. The item is the legendary "coin of wisdom", said to impart those who possess it with powerful knowledge of statistics. Legend has it that despite appearing to be a fair coin, the coin is biased, with an unequal probability of producing heads or tails. As the Professor of History could barely muster an addition if his life depended on it, he has reached out for collaborative assistance from our group to either confirm or deny the legend.

The coin flipping problem is of course a well-known problem from statistics, and when probabilities of heads or tails (or "probability of success" if we define success to be either heads or tails) are unequal the process can be modeled using the binomial distribution.

You have been tasked with estimating the probability that a flip of the coin produces tails (lets define tails as "success"). To do so, you collect a dataset of three sets of coin flips:
1. Your professor flips the coin five times, producing 1 tails and 4 heads, before realising he is late to an important meeting and handing the coin to a postdoc in the group.
2. The postdoc in the group flips the coin ten times, producing 5 tails and 5 heads, before realising it's time to collect the child from daycare and giving you the coin.
3. You flip the coin 100 times, producing 66 tails and 34 heads, before noticing how hungry you are and setting out to have dinner.

Because each person may have slight different biases in how they flip coins you decide not to pool all of the results, but instead to treat them as independent estimates under the binomial distribution. And for no particular reason, you decide to take the maximum likelihood computed under the binomial at fixed probabilities of tails/success rather than using analytical estimation or some kind of gradient optimisation.

Therefore, you calculate the likelihood of the three observations above when probability of tails ranges from 0.05, 0.1 ... 0.95 in increments of 0.05.

What is the maximum likelihood value of the probability of tails, p_t?

The likelihood that an arbitrary value p_t produced an observed set of coin flips can be calculated using the probability mass function of the binomial.


In [30]:
import scipy.stats as stats
# compute probability mass at each level of fairness for 
# the professor's coin flips
print("Professor:")
for i in range(1,20):
  print("\tp_t = "+str(i/20)+", likelihood="+str(stats.binom.pmf(1, 5, i/20)))

# now do the same for the postdoc's coin flips
print("Postdoc:")
for i in range(1,20):
  print("\tp_t = "+str(i/20)+", likelihood="+str(stats.binom.pmf(5, 10, i/20)))

# and for the student's coin flips
print("Student:")
for i in range(1,20):
  print("\tp_t = "+str(i/20)+", likelihood="+str(stats.binom.pmf(66, 100, i/20)))


Professor:
	p_t = 0.05, likelihood=0.20362656250000014
	p_t = 0.1, likelihood=0.32804999999999995
	p_t = 0.15, likelihood=0.39150468749999984
	p_t = 0.2, likelihood=0.4095999999999999
	p_t = 0.25, likelihood=0.39550781250000006
	p_t = 0.3, likelihood=0.3601499999999999
	p_t = 0.35, likelihood=0.31238593750000004
	p_t = 0.4, likelihood=0.2592000000000001
	p_t = 0.45, likelihood=0.20588906250000005
	p_t = 0.5, likelihood=0.15624999999999997
	p_t = 0.55, likelihood=0.11276718749999998
	p_t = 0.6, likelihood=0.0768
	p_t = 0.65, likelihood=0.048770312499999996
	p_t = 0.7, likelihood=0.02834999999999999
	p_t = 0.75, likelihood=0.014648437499999998
	p_t = 0.8, likelihood=0.006399999999999999
	p_t = 0.85, likelihood=0.0021515624999999994
	p_t = 0.9, likelihood=0.0004499999999999999
	p_t = 0.95, likelihood=2.968750000000011e-05
Postdoc:
	p_t = 0.05, likelihood=6.093524882812493e-05
	p_t = 0.1, likelihood=0.0014880347999999995
	p_t = 0.15, likelihood=0.008490855786328114
	p_t = 0.2, likelihood=0

## combining information from the separate trials

We have now obtained the likelihoods of different p_t for each of the three independent coin flip trials. How do we combine them? One approach would be to sum the (log) likelihoods across trials and take the maximum. Does it produce the correct result? Which trial contributes the most to the estimate of p_t?



## A further correspondence from the historian

The historian has come into contact with a colleague in Europe who is in possession of a letter written by the reverend that describes the manufacture of the coin. In it, the reverend indicates the shape and weight of the coin were designed to produce tails 60% of the time. How does that compare to your estimate? How does it compare to the student's estimate?

In [31]:
# scale factor
import numpy as np
prof_lk = [stats.binom.pmf(1, 5, i/20) for i in range(1,20)]
postdoc_lk = [stats.binom.pmf(5, 10, i/20) for i in range(1,20)]
student_lk = [stats.binom.pmf(66, 100, i/20) for i in range(1,20)]
avg_prof_lk = np.average(prof_lk)
avg_postdoc_lk = np.average(postdoc_lk)
avg_student_lk = np.average(student_lk)
print(avg_prof_lk,avg_postdoc_lk,avg_student_lk)
print(avg_prof_lk/avg_postdoc_lk, avg_prof_lk/avg_student_lk) # scaling factor

0.17434375 0.09569374787109375 0.010422094818650304
1.8218927973732764 16.728282848474226


In [32]:
0.010422094818650304*16.728282848474226

0.17434374999999996

In [33]:
postdoc_lk_scaled = np.array(postdoc_lk) * (avg_prof_lk/avg_postdoc_lk)
student_lk_scaled = np.array(student_lk) * (avg_prof_lk/avg_student_lk)

In [34]:
import pandas as pd
probabilities = [i/20 for i in range(1,20)]
prof_vals = pd.Series(prof_lk,probabilities)
postdoc_vals = pd.Series(postdoc_lk_scaled,probabilities)
student_vals = pd.Series(student_lk_scaled,probabilities)
postdoc_vals

0.05    0.000111
0.10    0.002711
0.15    0.015469
0.20    0.048142
0.25    0.106397
0.30    0.187508
0.35    0.279789
0.40    0.365578
0.45    0.426383
0.50    0.448356
0.55    0.426383
0.60    0.365578
0.65    0.279789
0.70    0.187508
0.75    0.106397
0.80    0.048142
0.85    0.015469
0.90    0.002711
0.95    0.000111
dtype: float64

In [35]:
collected_df = pd.concat([prof_vals,postdoc_vals,student_vals], axis=1)
collected_df

,0,1,2
0.05,0.203627,0.000111,2.301649e-59
0.10,0.328050,0.002711,2.701852e-40
0.15,0.391505,0.015469,1.620626e-29
0.20,0.409600,0.048142,3.634591e-22
0.25,0.395508,0.106397,1.008184e-16
0.30,0.360150,0.187508,1.624622e-12
0.35,0.312386,0.279789,3.427372e-09
0.40,0.259200,0.365578,1.515368e-06
0.45,0.205889,0.426383,1.869707e-04
0.50,0.156250,0.448356,7.663315e-03


In [36]:
collected_df_sum = collected_df.sum(axis=1)
collected_df_sum

0.05    0.203738
0.10    0.330761
0.15    0.406974
0.20    0.457742
0.25    0.501905
0.30    0.547658
0.35    0.592175
0.40    0.624779
0.45    0.632459
0.50    0.612270
0.55    0.654118
0.60    1.096168
0.65    1.702637
0.70    1.184157
0.75    0.307864
0.80    0.061247
0.85    0.017642
0.90    0.003161
0.95    0.000141
dtype: float64